In [2]:
#This is a continuation of the makemore names generator we made previously. As we found in the last study, the counting bigrams method gave similar results as the simple 1 layer nerual net,
# but we found that it would be incredibly hard to scale. Going from 1 character input to 2 character inputs would increase the possibilities from 27 to 27*27. If we had 3 characters,
# it would be 27*27*27, where n-character inputs is 27^n possibilities. Not only would this map be too large to handle, the normalized counts would all go to 0.

# Thankfully, we found a different approach. The neural network model is much easier to handle. Let's build out a more serious version of the net from the last study. For this, we will
# reference Bengio et al in their paper "A Neural Probabilistic Language Model". It's outdated technology, but a huge step forward in our journey towards name generation.

#https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [4]:
len(words)

32033

In [5]:
#build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [22]:
# Now lets build our dataset to train on. A clever way to do this is not to hardcode bigrams (like the last version), but make the input size variable. We will call this block_size

block_size = 3
X, Y = [], []
for w in words[:5]:

    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [23]:
X.shape, X.dtype

(torch.Size([32, 3]), torch.int64)

In [8]:
#In the Bengio et al paper, they havea  vocabulary of 17,000 words, and build vectors that represent those words in a 30-dimensional space. Since we are working with characters, we have 27.
# Let's try to fit them in a 2-dimensional space as a start. The paper refers to the parameter matrix as 'C', so we will re-use that.

C = torch.randn((27,2)) #row x column (27 x 2)

In [9]:
# Let's try an example of calling a value from the tensor. If we want the 5th row, we can simply write
C[5]

tensor([-1.5042, -0.9692])

In [14]:
# Last time we were using one-hot encoding. This takes the full C tensor and multiplies by a column vector of all 0s except that the 5th value is turned on (is a 1). This means
# all parts of C are multiplied by 0 except the fifth row which is multiplied by 1. This 'plucks out' the fifth C row.

F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-1.5042, -0.9692])

In [16]:
# Instead of doing all this multiplication, lets stick with the index notation. We can actually pull multiple values from C using lists or even tensors!
C[torch.tensor([4,5,6,7,7,7])]

tensor([[-2.0181, -0.3025],
        [-1.5042, -0.9692],
        [-1.0955, -0.7222],
        [-1.1462,  0.6276],
        [-1.1462,  0.6276],
        [-1.1462,  0.6276]])

In [27]:
# Ok this will blow your mind. You can feed in a 2 dimensional tensor to C and receive back a 3 dimensional tensor!!! 0.0
C[X].shape


torch.Size([32, 3, 2])

In [28]:
X[13,2]

tensor(1)

In [29]:
C[X][13,2]

tensor([ 0.4018, -0.7137])

In [30]:
C[1]

tensor([ 0.4018, -0.7137])

In [31]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])